### 네이버 영화 리뷰 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df =pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df =pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [3]:
print(train_df.shape,test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


#### 1. 데이터 전처리

- train data set

In [4]:
# 결측치가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [5]:
# 결측기 데이터 삭제
train_df.dropna(how='any',inplace=True)
train_df.shape

(149995, 3)

In [6]:
# 중복 데이터 확인
train_df.document.nunique()

146182

In [7]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'],inplace=True)
train_df.shape

(146182, 3)

In [8]:
# 데이터 분포
train_df.label.value_counts()

label
0    73342
1    72840
Name: count, dtype: int64

- 테스트 데이터 셋

In [9]:
test_df.dropna(how='any',inplace=True)
test_df.drop_duplicates(subset=['document'],inplace=True)
test_df.shape

(49157, 3)

In [10]:
test_df.label.value_counts()

label
1    24711
0    24446
Name: count, dtype: int64

#### 2. 텍스트 전처리

- 트레인 데이터 셋

In [11]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document= train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','',regex=True).str.strip()
train_df.head(3)

,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
1,3819312,흠포스터보고초딩영화줄오버연기조차가볍지않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [12]:
# 영어 데이터 -->''만 남게됨 --->np.nan 으로 대체한 후 제거
train_df.document.replace('',np.nan,inplace=True)
train_df.document.isna().sum()


789

In [13]:
train_df.dropna(how='any',inplace=True)
train_df.shape

(145393, 3)

- 테스트 데이터 set

In [14]:
# 한글 이외의 문자는 공백으로 처리하고 strip
test_df.document= test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]','',regex=True).str.strip()
test_df.document.replace('',np.nan,inplace=True)
test_df.dropna(how='any',inplace=True)
test_df.shape

(48852, 3)

In [15]:
train_df.to_csv('data/네이버영화리뷰_train_전처리완료.tsv',sep='\t',index=False)
test_df.to_csv('data/네이버영화리뷰_test_전처리완료.tsv',sep='\t',index=False)

##### 3. 한글 처리

In [16]:
from konlpy.tag import Okt
okt =Okt()

In [17]:
text ='흠포스터보고초딩영화줄오버연기조차가볍지않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [18]:
okt.morphs(text,stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [19]:
with open('data/불용어.txt') as st:
    lines = st.readlines()
print(len(lines))
print(lines[0])

100
이	VCP	0.018279601



In [20]:
stop_words =[]
for line in lines:
    word =line.split('\t')[0]
    stop_words.append(word)

In [21]:
stop_words =[line.split('\t')[0] for line in lines]
stop_words[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

- 형태소 분석기를 돌려서 나온 결과에서 stop_words 제거하기

In [22]:
morphs = okt.morphs(text,stem=True)
clean_morphs = [word for word in morphs if word not in stop_words]


In [23]:
# clean_morphs 문자열로 만들어 주어야 CountVectorizer의 입력으로 사용할 수 있음
clean_review =' '.join(clean_morphs)
clean_review


'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

한글 리뷰 str

형태소 분석기 list

불용어 제거 list

문자열로 변환 str

In [24]:
review='흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않구다'
morphs = okt.morphs(review,stem=True)
clean_morph_review =' '.join([word for word in morphs if word not in stop_words])
clean_morph_review

'흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다'

- train /test dataset 적용

In [26]:
%%time
X_train = []
for review in train_df.document:
    morphs = okt.morphs(review,stem=True)
    clean_morph_review =' '.join([word for word in morphs if word not in stop_words])
    X_train.append(clean_morph_review)


from tqdm import tqdm

X_train = []
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([word for word in morphs if word not in stop_words])
    X_train.append(clean_morph_review)

KeyboardInterrupt: 

In [ ]:
%%time
X_test =[]
for review in test_df.document:
    morphs = okt.morphs(review,stem=True)
    clean_morph_review =' '.join([word for word in morphs if word not in stop_words])
    X_test.append(clean_morph_review)

In [ ]:
y_train =train_df.label.values
y_test =test_df.label.values

##### 4. 피쳐 변환 + 모델 학습 /평가

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
pipeline =Pipeline([
    ('cvect',CountVectorizer()),
    ('nb',MultinomialNB())
])
%time pipeline.fit(X_train.y_train)

In [ ]:
pipeline.score(X_test,y_test)

#### 5. 실제 데이터 테스트

In [ ]:
reviews =[
    '이 영화 개꿀쨈 ㅋㅋㅋ',
    '이 영화 핵노잼 ㅠㅠㅠ',
    '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?',
    '와 개쩐다 정말 세계관 최강자들의 영화다'
]

In [ ]:
# 전처리 - 한글 이외의 문자는 공백으로 변환
import re
reviews =list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',s),reviews))
reviews

In [ ]:
# 형태소 분석 후 불용어 제거
processed_reviews =[]
for review in test_df.document:
    morphs = okt.morphs(reviews,stem=True)
    clean_morph_review =' '.join([word for word in morphs if word not in stop_words])
    processed_reviews.append(clean_morph_review)

In [ ]:
pipeline.predict(processed_reviews)

In [ ]:
pipeline.predict_proba(processed_reviews)